Predictor Bot : Predicting Crypto Prices for the next 10 candles on the 3 minute chart using LSTM 

In [47]:
#Packages Import
import math # Mathematical functions 
import numpy as np 
import pandas as pd 
from datetime import date, timedelta, datetime 
from pandas.plotting import register_matplotlib_converters
import matplotlib.pyplot as plt # Important package for visualization - we use this to plot the market data
import matplotlib.dates as mdates # Formatting dates
from sklearn.metrics import mean_absolute_error, mean_squared_error # Packages for measuring model performance / errors
from keras.models import Sequential # Deep learning library, used for neural networks
from keras.layers import LSTM, Dense, Dropout # Deep learning classes for recurrent and regular densely-connected layers
from keras.callbacks import EarlyStopping # EarlyStopping during model training
from sklearn.preprocessing import RobustScaler, MinMaxScaler # This Scaler removes the median and scales the data according to the quantile range to normalize the price data 
import seaborn as sns

In [48]:
pip install python-binance

Note: you may need to restart the kernel to use updated packages.


In [49]:
from datetime import datetime
from pandas_datareader.data import DataReader

In [50]:
# Data retrieval for cryptocurrencies BTCUSDT ETHUSDT XMRUSDT 
# Binance API credentials 
apikey = '2ZaEiNukiFtPHqnYdCGMANNimHdcCF0nvv4L9eYXMIdr4ovveQrca4NWKsZ7DxAy'
secretkey = '9zjousoyG0t2wblcGkm0tVGLBfz6Woo3paakGh49Vhj0IxrGGXl332b1KQaxlJKY'

from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
#API Initialization 
client = Client(apikey, secretkey)

In [51]:
pip install tf

Note: you may need to restart the kernel to use updated packages.


Real Forecast for BTCUSDT, ETHUSDT or XMRUSDT as inputs

In [56]:
#Actual Prediction

#Packages to load previously scaled data in array format for each crypto prediction in the below if statements
# load numpy array from npy file
import keras
from numpy import load

def prediction(crypto):
  #Choose between BTCUSDT ETHUSDT XMRUSDT
  if crypto == "BTCUSDT":
    #Fetch Last Day 3min Candles Data
    klines = client.get_historical_klines(crypto, Client.KLINE_INTERVAL_3MINUTE, "1 day ago UTC")

    pred_df = pd.DataFrame(klines)
    pred_df.columns = (['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume','Nb of Trade', 'TakerBuyBaseVolume', 'TakerBuyQuoteVolume','Ignored'])
    # hist_df.drop(labels = ['TakerBuyBaseVolume', 'TakerBuyQuoteVolume', 'Ignored', 'Quote Asset Volume'], inplace = True,axis = 1)
    pred_df['Close Time'] = pd.to_datetime(pred_df['Close Time']/1000, unit='s')
    # hist_df['Open Time'] = pd.to_datetime(hist_df['Open Time']/1000, unit='s')
    pred_df.drop(['Ignored', 'TakerBuyBaseVolume', 'TakerBuyQuoteVolume', 'Quote Asset Volume'], inplace= True, axis = 1)

    #load scaled xmr array
    np_scaled = load('scaled/btc_np_scaled.npy')

    pred_df = pred_df.apply(pd.to_numeric)
    pred = np_scaled[-51:-1,:].reshape(1,50,5)

    # pred_df.to_csv('5H_avg_pred_df')
    # pred_df
    model = keras.models.load_model('Models\BTC_Model_3MIN.h5')
    btc_y_pred_scaled = model(pred)
    btc_y_pred = scaler_pred.inverse_transform(btc_y_pred_scaled)

    print(btc_y_pred)

  elif crypto == "XMRUSDT":

    #Fetch Last Day 3MINUTE Candles Data
    klines = client.get_historical_klines(crypto, Client.KLINE_INTERVAL_3MINUTE, "1 day ago UTC")

    pred_df = pd.DataFrame(klines)
    pred_df.columns = (['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume','Nb of Trade', 'TakerBuyBaseVolume', 'TakerBuyQuoteVolume','Ignored'])
    # hist_df.drop(labels = ['TakerBuyBaseVolume', 'TakerBuyQuoteVolume', 'Ignored', 'Quote Asset Volume'], inplace = True,axis = 1)
    pred_df['Close Time'] = pd.to_datetime(pred_df['Close Time']/1000, unit='s')
    # hist_df['Open Time'] = pd.to_datetime(hist_df['Open Time']/1000, unit='s')
    pred_df.drop(['Ignored', 'TakerBuyBaseVolume', 'TakerBuyQuoteVolume', 'Quote Asset Volume'], inplace= True, axis = 1)

    #load scaled xmr array
    np_scaled = load(r'scaled\xmr_np_scaled.npy')

    xmr_pred_df = pred_df.apply(pd.to_numeric)
    xmr_pred = np_scaled[-51:-1,:].reshape(1,50,5)

    # pred_df.to_csv('5H_avg_pred_df')
    # pred_df
    xmr_model = keras.models.load_model('Models\XMR_Model_3MIN.h5')
    xmr_y_pred_scaled = xmr_model(xmr_pred)
    xmr_y_pred = scaler_pred.inverse_transform(xmr_y_pred_scaled)

    print(xmr_y_pred)
      
  elif crypto == "ETHUSDT":
    #Fetch Last Day  3min Candles Data
    klines = client.get_historical_klines(crypto, Client.KLINE_INTERVAL_3MINUTE, "1 day ago UTC")

    pred_df = pd.DataFrame(klines)
    pred_df.columns = (['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume','Nb of Trade', 'TakerBuyBaseVolume', 'TakerBuyQuoteVolume','Ignored'])
    # hist_df.drop(labels = ['TakerBuyBaseVolume', 'TakerBuyQuoteVolume', 'Ignored', 'Quote Asset Volume'], inplace = True,axis = 1)
    pred_df['Close Time'] = pd.to_datetime(pred_df['Close Time']/1000, unit='s')
    # hist_df['Open Time'] = pd.to_datetime(hist_df['Open Time']/1000, unit='s')
    pred_df.drop(['Ignored', 'TakerBuyBaseVolume', 'TakerBuyQuoteVolume', 'Quote Asset Volume'], inplace= True, axis = 1)
    # hist_df['Open Time'] = hist_df.index

    #load scaled xmr array
    np_scaled = load(r'scaled/eth_np_scaled.npy')
    eth_pred_df = pred_df.apply(pd.to_numeric)
    eth_pred = np_scaled[-51:-1,:].reshape(1,50,5)

    eth_model = keras.models.load_model('Models\ETH_Model_3MIN.h5')
    eth_y_pred_scaled = eth_model(eth_pred)
    eth_y_pred = scaler_pred.inverse_transform(eth_y_pred_scaled)

    print(eth_y_pred)


In [59]:
prediction("ETHSDT")